In [62]:
# !pip install datasets

In [63]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoModelForSeq2SeqLM
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from datasets import Dataset

In [64]:
data = pd.read_csv('dataset.csv')  # Make sure this CSV file has 'disease' and 'symptoms' columns
data = data[['disease', 'symptoms']]
data.head()

,disease,symptoms
0,flu,"fever,cough,sore throat,runny or stuffy nose,m..."
1,bronchitis,"cough,mucus production,shortness of breath,che..."
2,pneumonia,"fever,cough,shortness of breath,chest pain,fat..."
3,heart attack,"chest pain,shortness of breath,nausea,vomiting..."
4,stroke,"sudden weakness,numbness on one side of the bo..."


In [65]:
data.columns = ["disease", "symptoms"]  # Ensure correct column names if needed
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['disease'])  # Labels from 0 to 99

In [66]:
tokenizer = AutoTokenizer.from_pretrained("describeai/gemini")
model = AutoModelForSeq2SeqLM.from_pretrained("describeai/gemini")

In [67]:
print(tokenizer.tokenize(data['symptoms'][0]))
data['symptoms'][0]

['▁fever', ',', 'c', 'ough', ',', 's', 'or', 'e', '▁throat', ',', 'run', 'n', 'y', '▁or', '▁stuff', 'y', '▁nose', ',', 'mus', 'cle', '▁', 'aches', ',', 'head', 'ache', ',', 'fat', 'igue']


'fever,cough,sore throat,runny or stuffy nose,muscle aches,headache,fatigue'

In [68]:
# train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

temp_texts, test_texts, temp_labels, test_labels = train_test_split(data['symptoms'], data['label'], test_size=0.2, random_state=42)
train_texts, val_texts, train_labels, val_labels = train_test_split(temp_texts, temp_labels, test_size=0.25, random_state=42)

In [69]:
def tokenize_texts(texts):
    return tokenizer(
        texts.tolist(),
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

# Prepare encodings for both train and validation sets
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=128)

# Convert labels to tensors
# train_labels = torch.tensor(train_data['label'].values)
# val_labels = torch.tensor(val_data['label'].values)

In [70]:
class DiseaseSymptomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

In [73]:
# train_dataset = DiseaseSymptomDataset(train_encodings, train_labels)
# val_dataset = DiseaseSymptomDataset(val_encodings, val_labels)

train_dataset = Dataset.from_dict({'input_ids': train_encodings['input_ids'], 'attention_mask': train_encodings['attention_mask'], 'labels': list(train_labels)})
val_dataset = Dataset.from_dict({'input_ids': val_encodings['input_ids'], 'attention_mask': val_encodings['attention_mask'], 'labels': list(val_labels)})


# Fine-tuning Setup
# training_args = TrainingArguments(
#     output_dir='./results',
#     eval_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     num_train_epochs=3,
#     weight_decay=0.01,
# )

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

# Define a compute metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted"),
    }

# Trainer for fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [74]:
trainer.train()

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
model.save_pretrained("fine_tuned_gemini_disease_model")
tokenizer.save_pretrained("fine_tuned_gemini_disease_model")

In [ ]:
preds_output = trainer.predict(val_dataset)
accuracy = accuracy_score(val_labels, preds_output.predictions.argmax(-1))
f1 = f1_score(val_labels, preds_output.predictions.argmax(-1), average="weighted")

print(f"Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")

# Save original model for comparison if needed
model.save_pretrained("original_gemini_model")